In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as PLT
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, Model,Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D,ZeroPadding2D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import os,glob,random,shutil
from pathlib import Path
from tensorflow.keras.optimizers.legacy import Adam
os.path.expanduser('/home/masoud/.keras/')

'/home/masoud/.keras/'

In [5]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
data_path = os.path.join('__file__', '../../../../datasets/15-Scene/data/')
train_test_path = os.path.join('__file__', '../../../../datasets/15-Scene/train_test/')
train_path = os.path.join(train_test_path, 'train/')
test_path = os.path.join(train_test_path, 'test/')

In [7]:
train_count = 100

if os.path.isdir(train_test_path):
    shutil.rmtree(train_test_path)
Path(f'{train_test_path}').mkdir(parents=True, exist_ok=True)

for category in os.listdir(data_path):
    images = glob.glob(f'{data_path}/{category}/*')
    random.shuffle(images)
    
    # train data
    Path(f'{train_path}/{category}/').mkdir(parents=True, exist_ok=True)
    
    for img in images[:train_count]:
        shutil.copyfile(img, f'{train_path}/{category}/{img.split("/")[-1]}')
        
    # test data
    Path(f'{test_path}/{category}/').mkdir(parents=True, exist_ok=True)
    for img in images[train_count:]:
        shutil.copyfile(img, f'{test_path}/{category}/{img.split("/")[-1]}')

In [8]:
image_generator = ImageDataGenerator()
batch_size = 15

generator = image_generator.flow_from_directory(
    train_path,
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(150,150)
)

generator_test = image_generator.flow_from_directory(
    test_path,
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(150,150)
)

Found 1500 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.


# Default laerning rate 0.001

In [9]:
model = Sequential([
 
    
    Conv2D(128, 3, strides=1, activation='relu', padding='same'),
#     BatchNormalization(axis=-1),
    MaxPool2D(pool_size=16, strides=2),
    
    Conv2D(64, 3, strides=1, activation='relu'),
#     BatchNormalization(axis=-1),
    MaxPool2D(pool_size=8, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
#     BatchNormalization(axis=-1),
    MaxPool2D(pool_size=4, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
#     BatchNormalization(axis=-1),
    MaxPool2D(pool_size=2, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
#     BatchNormalization(axis=-1),
    MaxPool2D(pool_size=2, strides=2),
    
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

2023-02-24 22:03:26.502835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 22:03:26.503363: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 22:03:26.503506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 22:03:26.503566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

In [11]:
history = model.fit(
    generator,
    validation_data=generator_test,
    epochs=20,
)

Epoch 1/20


2023-02-24 22:33:07.570727: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 164.79MiB (rounded to 172800000)requested by op sequential/conv2d/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-24 22:33:07.570938: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-02-24 22:33:07.570977: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 53, Chunks in use: 52. 13.2KiB allocated for chunks. 13.0KiB in use in bin. 2.3KiB client-requested in use in bin.
2023-02-24 22:33:07.570999: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.2KiB allocated for chunks. 3.2KiB in use in bin. 2.7KiB client-requested in use in bin.
2023-02-24 22:33:07.571018: I tensorflow/ts

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "/home/masoud/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/masoud/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/masoud/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/masoud/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/masoud/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/masoud/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/masoud/anaconda3/lib/python3.9/asyncio/base_even

In [ ]:
PLT.plot(history.history['accuracy'],color='green')
PLT.plot(history.history['val_accuracy'],color='red')

In [ ]:
# model.save('15-scene')
# model = models.load_model('15-scene')

# Diffrent Learning rate

In [ ]:
model = Sequential([
 
    
    Conv2D(128, 3, strides=1, activation='relu', padding='same'),
    MaxPool2D(pool_size=16, strides=2),
    
    Conv2D(64, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=8, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=4, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    
    
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.03), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    generator,
    validation_data=generator_test,
    epochs=20,
)

In [ ]:
PLT.plot(history.history['accuracy'],color='green')
PLT.plot(history.history['val_accuracy'],color='red')

In [ ]:
model = Sequential([
 
    
    Conv2D(128, 3, strides=1, activation='relu', padding='same'),
    MaxPool2D(pool_size=16, strides=2),
    
    Conv2D(64, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=8, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=4, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    
    
    Conv2D(32, 3, strides=1, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    
    
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    generator,
    validation_data=generator_test,
    epochs=20,
)

In [ ]:
PLT.plot(history.history['accuracy'],color='green')
PLT.plot(history.history['val_accuracy'],color='red')

# ResNet Evaluation

In [ ]:
resnet152 = ResNet152(weights='imagenet', input_shape=generator.image_shape, include_top=False)

In [ ]:
resnet_model = models.Sequential()
resnet_model.add(resnet152)
resnet_model.add(Flatten())
resnet_model.add(Dense(15, activation='softmax'))
resnet_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
resnet_history = resnet_model.fit(
    generator,
    validation_data=generator_test,
    epochs=200,
)

In [ ]:
PLT.plot(resnet_history.history['accuracy'],color='green')
PLT.plot(resnet_history.history['val_accuracy'],color='red')